In [1]:
import re
import numpy as np
import string

In [2]:
file = open("kanye_verses.txt", "r", encoding = "utf8")
text = file.read()
text = text.replace("\n\n", "\n")

In [3]:
def clean_lyric(txt):
    return re.sub("[^a-z' ]", "", txt)

In [4]:
lyrics = text.lower().split("\n")
lyrics = np.unique(lyrics)[1:].tolist()
cleaned_lyrics = [clean_lyric(lyric) for lyric in lyrics]

In [5]:
def create_sequences(lyric, seq_len):
    sequences = []    
    if len(lyric.split()) <= seq_len:
        return [lyric]
    
    for itr in range(seq_len, len(lyric.split())):
        curr_seq = lyric.split()[itr - seq_len:itr + 1]
        sequences.append(" ".join(curr_seq))
    return sequences

In [6]:
raw_sequences = [create_sequences(cleaned_lyric, 2) for cleaned_lyric in cleaned_lyrics]

In [7]:
sequences = np.unique(np.array(sum(raw_sequences, []))).tolist()

In [24]:
uniq_words = np.unique(np.array(" ".join(sequences).split(" ")))
uniq_words_idx = np.arange(uniq_words.size)

bag_of_words_dict = zip(uniq_words_idx.tolist(), uniq_words.tolist())